# Currency Crash Prediction - Signal Generation

Generate R-Zone signals and identify crash events

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signal_generator import CurrencyCrashPredictor

sns.set_style('darkgrid')
%matplotlib inline

## 1. Load Data and Generate Signals

In [ ]:
predictor = CurrencyCrashPredictor()
predictor.load_data()
signals = predictor.generate_signals()

## 2. Visualize R-Zone Signals

In [ ]:
r_zone = signals['r_zone']
crashes = signals['crashes']

# Plot R-Zone timeline for each currency
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, currency in enumerate(r_zone.columns[:9]):
    ax = axes[i]
    
    # Plot R-Zone periods
    r_zone_dates = r_zone[r_zone[currency] == 1].index
    crash_dates = crashes[crashes[currency] == 1].index
    
    ax.scatter(r_zone_dates, [1]*len(r_zone_dates), 
              color='orange', label='R-Zone', s=50, alpha=0.7)
    ax.scatter(crash_dates, [0]*len(crash_dates), 
              color='red', label='Crash', s=50, alpha=0.7)
    
    ax.set_title(f'{currency}')
    ax.set_ylim(-0.5, 1.5)
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['Crash', 'R-Zone'])
    ax.legend()

plt.tight_layout()
plt.show()

## 3. Statistics Summary

In [ ]:
print("Crash Prediction Statistics:")
print(predictor.statistics)

# Overall statistics
avg_crash_prob_rzone = predictor.statistics['crash_prob_in_rzone'].mean()
avg_baseline_prob = predictor.statistics['baseline_crash_prob'].mean()

print(f"\nAverage Crash Probability in R-Zone: {avg_crash_prob_rzone:.2%}")
print(f"Average Baseline Crash Probability: {avg_baseline_prob:.2%}")
print(f"Probability Ratio: {avg_crash_prob_rzone/avg_baseline_prob:.2f}x")

## 4. Visualize Feature Distributions

In [ ]:
delta_i = signals['delta_i']
delta_fx = signals['delta_fx']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Δi distribution
delta_i.plot(kind='box', ax=axes[0])
axes[0].set_title('Δi Distribution (6-month rate change)')
axes[0].set_ylabel('Rate Change')
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)

# ΔFX distribution
delta_fx.plot(kind='box', ax=axes[1])
axes[1].set_title('ΔFX Distribution (6-month log change)')
axes[1].set_ylabel('Log Change')
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 5. R-Zone vs Crash Scatter Plot

In [ ]:
# Example for EUR
currency = 'EUR'

if currency in delta_i.columns:
    df = pd.DataFrame({
        'delta_i': delta_i[currency],
        'delta_fx': delta_fx[currency],
        'r_zone': r_zone[currency],
        'crash': crashes[currency]
    }).dropna()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot all points
    scatter = ax.scatter(df['delta_i'], df['delta_fx'], 
                        c=df['crash'], cmap='RdYlGn_r', 
                        s=100, alpha=0.6, edgecolors='black')
    
    # Highlight R-Zone
    r_zone_points = df[df['r_zone'] == 1]
    ax.scatter(r_zone_points['delta_i'], r_zone_points['delta_fx'],
              s=200, facecolors='none', edgecolors='blue', 
              linewidths=2, label='R-Zone')
    
    ax.set_xlabel('Δi (6-month rate change)', fontsize=12)
    ax.set_ylabel('ΔFX (6-month log change)', fontsize=12)
    ax.set_title(f'{currency}: R-Zone and Crash Events', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.colorbar(scatter, label='Crash (1=Yes, 0=No)')
    plt.tight_layout()
    plt.show()

## 6. Lead Time Analysis

In [ ]:
# Calculate average lead time from R-Zone to crash
lead_times = []

for currency in r_zone.columns:
    r_zone_dates = r_zone[r_zone[currency] == 1].index
    
    for r_date in r_zone_dates:
        # Find next crash within 6 months
        future_dates = pd.date_range(r_date, periods=7, freq='M')[1:]
        future_crashes = crashes.loc[crashes.index.isin(future_dates), currency]
        
        if future_crashes.sum() > 0:
            crash_date = future_crashes[future_crashes == 1].index[0]
            lead_time = (crash_date - r_date).days / 30  # Convert to months
            lead_times.append(lead_time)

if lead_times:
    plt.figure(figsize=(10, 6))
    plt.hist(lead_times, bins=20, edgecolor='black', alpha=0.7)
    plt.axvline(np.mean(lead_times), color='r', linestyle='--', 
               label=f'Mean: {np.mean(lead_times):.1f} months')
    plt.xlabel('Lead Time (months)')
    plt.ylabel('Frequency')
    plt.title('Lead Time from R-Zone Entry to Crash')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Average Lead Time: {np.mean(lead_times):.2f} months")
    print(f"Median Lead Time: {np.median(lead_times):.2f} months")